### Regularized (banded) CV regression workflow for Neuroscout
This notebook implements an encoding model for a single subject using Regularized Ridge Regression, as implemented in https://github.com/gallantlab/himalaya. In Neuroscout, this same pipeline should be run for all subjects.
- Input needed from the user
    - Define datasets (independent model fitting for all datasets)
    - Define cross-validation strategy
        - Across runs
        - Within runs
    - Define estimator
    - Define preprocessing steps (e.g., scaling?)
    - Define bands
    - Pass parameters
    - Output: scores, parameters, predicted time series
- Define outputs

<b> To do<b>:
- Decide which outputs to store
- Implement

In [1]:
import pyns
import pandas as pd
import nibabel as nib
import numpy as np
import glob
from copy import deepcopy
from pathlib import Path

/home/alejandro/anaconda3/lib/python3.7/site-packages/nilearn/datasets/__init__.py:89: FutureWarning: Fetchers from the nilearn.datasets module will be updated in version 0.9 to return python strings instead of bytes and Pandas dataframes instead of Numpy arrays.
  "Numpy arrays.", FutureWarning)


In [2]:
api = pyns.Neuroscout()

### Get fMRI data from datalad
Let's retrieve data for a couple of subjects from Budapest.
Neuroscout dataset can be found under the `neuroscout-datasets` organization: https://github.com/neuroscout-datasets

In [3]:
dataset_path = Path('/media/neuroscout-data/neuroscout/datasets/neuroscout-datasets/Budapest')

In [4]:
%%capture
# !datalad get /media/neuroscout-data/neuroscout/datasets/neuroscout-datasets/Budapest/fmriprep/sub-sid000005/func/*-preproc_bold.nii.gz
#!datalad get ds003017/fmriprep/sub-sid000007/func/*-preproc_bold.nii.gz
#!datalad get ds003017/fmriprep/sub-sid000009/func/*-preproc_bold.nii.gz

In [5]:
TR = 1
subject = 'sid000005'
sub_id = 'Budapest'

NOTE: Replace this with pybids logic

In [6]:
def _make_outputs(dataset_path, sub_id=None, TR=1):
    arrays = []
    fs = dataset_path.glob(f'fmriprep/sub-{sub_id}/func/*-preproc_bold.nii.gz')
    fs = [str(f) for f in fs]
    runs = sorted(set([f.split('/')[-1].split('_')[2].split('-')[1] for f in fs]))
    resampling_ts = []
    for r_ix, r in enumerate(runs):
        img_file = [f for f in fs if f'run-{r}' in f][0]
        data = np.asanyarray(nib.load(img_file).dataobj)
        run_y = data.reshape([data.shape[0] * data.shape[1] * data.shape[2], data.shape[3]]).T
        arrays.append(run_y)
        resampling_ts.append(np.arange(0, TR*run_y.shape[0], step=TR))
    return np.vstack(arrays), resampling_ts

In [7]:
Y, resampling_ts = _make_outputs(dataset_path, sub_id=subject, TR=TR)

In [8]:
Y.shape

(3052, 565812)

In [9]:
sum([t.shape[0] for t in resampling_ts])

3052

### Build input matrix
Let's retrieve predictor events for multiple sets of predictors. \
For now, let's pick three sets: <b>Audioset</b> + <b>MFCC</b> + <b>mel</b> features (plus some confounds).

In [10]:
audioset = ['as-Music',
            'as-Animal',
            'as-Whistling',
            'as-Vehicle',
            'as-Wild animals',
            'as-Thunderstorm',
            'as-Noise',
            'as-Fire',
            'as-Water',
            'as-Wind',
            'as-Glass',
            'as-Wood',
            'as-Silence',
            'as-Mechanisms',
            'as-Alarm',
            'as-Hands',
            'as-Tools',
            'as-Speech',
            'as-Explosion',
            'as-Engine',
            'as-Liquid',
            'as-Musical instrument']
mfccs = [f'mfcc_{i}' for i in range(20)]
mel = [f'mel_{i}' for i in range(64)]
confounds = ['rot_x', 'rot_y', 'rot_z', 'trans_x', 'trans_y', 'trans_z',
             'a_comp_cor_00', 'a_comp_cor_01', 'a_comp_cor_02',
             'a_comp_cor_03','a_comp_cor_04','a_comp_cor_05']

Defining a function that does naive resampling of predictor events to TR.

Defining a function that takes a list of lists (`predictor_sets`), a dataset name, and a subject id and returns:
- `mats`, a list of pandas DataFrame (one per predictor set) of shape $n\_TRs x n\_features$;
- `run_index`, a list of the same length as `mats`, with run_ids for each row in `mats`.
Both `mats` and `run_index` are obtained by concatenating multiple runs. \
Each element of `predictor_sets` is a list of predictor names (e.g., `as-Speech`, `as-Music`). \
We also pass `resampling_ts` - not relevant for Neuroscout implementation, only needed for **ad hoc** resampling in this context.

In [11]:
from bids.variables import SparseRunVariable, BIDSRunVariableCollection
from bids.variables.entities import RunInfo
import math

TODO: Given a combined subject_df, group_by predictor / run, resample to TR, and combin

In [92]:
def fetch_predictors_as_collections(predictor_name, dataset_name, **entities):
    all_df = api.predictor_events.get(
        predictor_name=predictor_name, dataset_name=dataset_name, output_type='df', **entities)
    all_df = all_df.rename(columns={'number': 'run'})
    
    all_run_info = {}
    for r_id in all_df.run_id.unique():
        resp = api.runs.get(r_id)
        all_run_info[r_id] = {
            'duration': resp['duration'],
            'tr': api.tasks.get(resp['task'])['TR']
        }
        
    variables = []
    for (run_id, predictor_id), df in all_df.groupby(['run_id', 'predictor_id']):
        df = df.sort_values('onset')

        name = df['predictor_name'].iloc[0]
        meta_cols = ['subject', 'session', 'run', 'acquisition', 'run_id']
        entities = {}
        for j in meta_cols:
            val = df[j].iloc[0]
            if val:
                entities[j] = val
            else:
                # Drop column from DF
                df = df.drop(columns=j)
        meta_cols = [m for m in meta_cols if m in df]

        # TODO: Fetch real number of volumes, or allowing passing it in
        this_run_info = all_run_info[entities['run_id']]
        this_run_info['n_vols'] = math.ceil(this_run_info['duration'] / this_run_info['tr'])

        run_info = RunInfo(**this_run_info, entities=entities, image=None)
        df = df[['onset', 'duration', 'value'] + meta_cols].rename(columns={'value': 'amplitude'})

        try:
            df['amplitude'] = pd.to_numeric(df['amplitude'])
        except ValueError:
            pass

        variables.append(SparseRunVariable(name, df, run_info, 'events'))
            
        
    return BIDSRunVariableCollection(variables=variables)

In [103]:
collection = fetch_predictors_as_collections(
    predictor_name=['as-Music', 'as-Speech'], dataset_name='Budapest', subject='sid000005')

In [114]:
def _make_input_matrices(collection):
    resampled = collection.to_dense().resample('TR')
    df = resampled.to_df()
    
    keep = list(collection.variables.keys()) + ['onset', 'duration']
    mat = df[keep]
    metadata = df.drop(columns=keep)
    
    return mat, metadata

In [115]:
mats, idx = _make_input_matrices(collection)

In [16]:
mats[0].shape

(3052, 2)

### Preprocessing and model fitting

Run the encoding model with lots of comments and printing.

In [18]:
from sklearn.model_selection import KFold, GroupKFold, PredefinedSplit
from himalaya.ridge import GroupRidgeCV
from himalaya.scoring import correlation_score

Cross-validated model fitting, prediction, and scoring loosely based on scikit-learn's [`cross_val_score`](https://scikit-learn.org/stable/modules/generated/sklearn.model_selection.cross_val_score.html). Returns a `results` dictionary with `'coefficients'`, `'test_predictions'`, and `'test_scores'` keys containing lists of numpy arrays for each outer cross-validation fold.

In [19]:
def _model_cv(estimator, X, y, groups=None,
              scoring=correlation_score, cv=None,
              inner_cv=None, confounds=None, split=False):

    # Container for results
    results = {
        'coefficients': [],
        'test_predictions': [],
        'test_scores': []}
    
    # Extract number of samples for convenience
    n_samples = y.shape[0]
    
    # Keep count of confounds (if provided) for later
    n_confounds = confounds.shape[1] if confounds is not None else 0
    
    # If confounds are provided, stack them onto X model
    if confounds is not None:
        X = X + [confounds] if type(X) == list else np.hstack((X, confounds))
    
    # Set default cross-validation to KFold if not specified
    cv = KFold() if not cv else cv
    
    # Loop through outer cross-validation folds
    for train, test in cv.split(np.arange(n_samples), groups=groups):
        
        # Get training model for list of model bands
        X_train = [x[train] for x in X] if type(X) == list else X[train]
        X_test = [x[test] for x in X] if type(X) == list else X[test]
        
        # Create inner cross-validation loop if specified
        if inner_cv:
            
            # Split inner cross-validation with groups if supplied
            inner_groups = np.array(groups)[train] if groups else groups
            inner_splits = inner_cv.split(np.arange(n_samples)[train],
                                          groups=inner_groups)
            
            # Update estimator with inner cross-validator
            estimator.set_params(cv=inner_splits)
            print(np.unique(inner_groups))
        
        # Fit the regression model on training data
        estimator.fit(X_train, y[train])
        
        # Zero out coefficients for confounds if provided
        if confounds is not None:
            estimator.coef_[-n_confounds:] = 0
        
        # Compute predictions with optional splitting by band
        test_prediction = estimator.predict(X_test, split=split)
        
        # Test scores should also optionally split by band
        test_score = scoring(y[test], test_prediction)
        
        # Populate results dictionary
        results['coefficients'].append(estimator.coef_)
        results['test_predictions'].append(test_prediction)
        results['test_scores'].append(test_score)
        
    return results

In [20]:
# Convert input data to numpy and fill NaNs
X = [mat.fillna(0).to_numpy() for mat in mats]
y = Y[:, :100]

# Optionally create some dummy confounds
confounds = np.random.randn(y.shape[0], 2)

# Default estimator should be GroupRidgeCV
estimator = GroupRidgeCV(groups='input')

# Default cross-validation should be leave-one-run-out
n_runs = len(np.unique(idx))
cv = GroupKFold(n_splits=n_runs)
inner_cv = GroupKFold(n_splits=n_runs - 1)

In [21]:
# Run model with specified cross-validation, groups, confounds, and split outputs
results = _model_cv(estimator, X, y, cv=cv, inner_cv=inner_cv, groups=idx, confounds=confounds, split=True)

[1433 1434 1435 1436]
[........................................] 100% | 2.25 sec | 100 random sampling with cv | 
[1433 1434 1435 1437]
[........................................] 100% | 2.56 sec | 100 random sampling with cv | 
[1434 1435 1436 1437]
[........................................] 100% | 2.71 sec | 100 random sampling with cv | 
[1433 1434 1436 1437]
[........................................] 100% | 2.81 sec | 100 random sampling with cv | 
[1433 1435 1436 1437]
[........................................] 100% | 2.58 sec | 100 random sampling with cv | 


In [22]:
results = _model_cv(estimator, X, y, cv=cv, inner_cv=inner_cv, groups=idx, confounds=confounds)

[1433 1434 1435 1436]
[........................................] 100% | 2.01 sec | 100 random sampling with cv | 
[1433 1434 1435 1437]
[........................................] 100% | 2.28 sec | 100 random sampling with cv | 
[1434 1435 1436 1437]
[........................................] 100% | 2.37 sec | 100 random sampling with cv | 
[1433 1434 1436 1437]
[........................................] 100% | 2.49 sec | 100 random sampling with cv | 
[1433 1435 1436 1437]
[........................................] 100% | 2.77 sec | 100 random sampling with cv | 


In [23]:
results = _model_cv(estimator, X, y, cv=KFold(), inner_cv=KFold())

[None]
[........................................] 100% | 2.57 sec | 100 random sampling with cv | 
[None]
[........................................] 100% | 2.67 sec | 100 random sampling with cv | 
[None]
[........................................] 100% | 2.38 sec | 100 random sampling with cv | 
[None]
[........................................] 100% | 2.73 sec | 100 random sampling with cv | 
[None]
[........................................] 100% | 2.68 sec | 100 random sampling with cv | 


In [127]:
results = _model_cv(estimator, X, y, cv=KFold())

[........................................] 100% | 0.10 sec | 100 random sampling with cv | 
[..................................      ] 86% | 0.10 sec | 100 random sampling with cv | 

/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/himalaya/backend/numpy.py:43: RuntimeWarning: Mean of empty slice.
  return array.mean(axis, dtype=np.float64,
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/himalaya/backend/numpy.py:43: RuntimeWarning: Mean of empty slice.
  return array.mean(axis, dtype=np.float64,
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


[........................................] 100% | 0.12 sec | 100 random sampling with cv | 
[........................................] 100% | 0.11 sec | 100 random sampling with cv | 
[..................................      ] 85% | 0.09 sec | 100 random sampling with cv | 

/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/himalaya/backend/numpy.py:43: RuntimeWarning: Mean of empty slice.
  return array.mean(axis, dtype=np.float64,
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/himalaya/backend/numpy.py:43: RuntimeWarning: Mean of empty slice.
  return array.mean(axis, dtype=np.float64,
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


[........................................] 100% | 0.10 sec | 100 random sampling with cv | 
[........................................] 100% | 0.10 sec | 100 random sampling with cv | 


/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/himalaya/backend/numpy.py:43: RuntimeWarning: Mean of empty slice.
  return array.mean(axis, dtype=np.float64,
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


In [128]:
results = _model_cv(estimator, X, y, inner_cv=KFold())

[None]
[........................................] 100% | 2.77 sec | 100 random sampling with cv | 
[None]
[........................................] 100% | 2.67 sec | 100 random sampling with cv | 
[None]
[........................................] 100% | 2.78 sec | 100 random sampling with cv | 
[None]
[........................................] 100% | 2.69 sec | 100 random sampling with cv | 
[None]
[........................................] 100% | 2.65 sec | 100 random sampling with cv | 


In [117]:
results = _model_cv(estimator, X, y, groups=idx)

[........................................] 100% | 0.10 sec | 100 random sampling with cv | 
[...................................     ] 89% | 0.09 sec | 100 random sampling with cv | 

/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/himalaya/backend/numpy.py:43: RuntimeWarning: Mean of empty slice.
  return array.mean(axis, dtype=np.float64,
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/himalaya/backend/numpy.py:43: RuntimeWarning: Mean of empty slice.
  return array.mean(axis, dtype=np.float64,
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


[........................................] 100% | 0.10 sec | 100 random sampling with cv | 
[........................................] 100% | 0.12 sec | 100 random sampling with cv | 
[............................            ] 71% | 0.08 sec | 100 random sampling with cv | 

/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/himalaya/backend/numpy.py:43: RuntimeWarning: Mean of empty slice.
  return array.mean(axis, dtype=np.float64,
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/himalaya/backend/numpy.py:43: RuntimeWarning: Mean of empty slice.
  return array.mean(axis, dtype=np.float64,
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


[........................................] 100% | 0.11 sec | 100 random sampling with cv | 
[........................................] 100% | 0.11 sec | 100 random sampling with cv | 


/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/himalaya/backend/numpy.py:43: RuntimeWarning: Mean of empty slice.
  return array.mean(axis, dtype=np.float64,
/Users/snastase/opt/miniconda3/envs/datalad/lib/python3.9/site-packages/numpy/core/_methods.py:181: RuntimeWarning: invalid value encountered in true_divide
  ret = um.true_divide(


In [147]:
# Using single-band (non-banded) model with sklearn RidgeCV
from sklearn.linear_model import RidgeCV
results = _model_cv(RidgeCV(), X[0], y, cv=cv, inner_cv=inner_cv, groups=idx)

[1433 1434 1435 1436]
[1433 1434 1435 1437]
[1434 1435 1436 1437]
[1433 1434 1436 1437]
[1433 1435 1436 1437]


### Handling outputs

In [ ]:
### 

### Validate against other workflows

In [ ]:
### 